<a href="https://colab.research.google.com/github/msavazzi28/portifolio/blob/main/Gerador_de_bases_fake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.9 MB/s eta 0:00:00


In [ ]:
import faker
gerador = faker.Faker('PT-BR')

In [ ]:
from faker import Faker
import pandas as pd
import random

# Inicializando o Faker
fake = Faker()

# Parâmetros
num_linhas = 1000           # Total de registros (linhas)
num_produtos_unicos = 100   # Total de produtos únicos

# Categorias
categorias = ['Alimentos', 'Limpeza', 'Higiene', 'Bebidas', 'Utilidades']

# Função para gerar EAN válido (13 dígitos)
def gerar_ean():
    return fake.ean(length=13)

# Função para gerar uma posição no estoque no formato A12N03
def gerar_posicao():
    corredor = random.choice("ABCDEFGH")
    prateleira = random.randint(1, 14)
    nivel = random.randint(1, 4)
    return f"{corredor}-{nivel:02}-{prateleira:02}"

# Gerar produtos únicos com nome, EAN, categoria e preço fixos
produtos_unicos = []
for _ in range(num_produtos_unicos):
    produto = {
        "nome_produto": fake.word().capitalize() + " " + fake.word().capitalize(),
        "ean": gerar_ean(),
        "categoria": random.choice(categorias),
        "preco": round(random.uniform(10, 100), 2)
    }
    produtos_unicos.append(produto)

# Criar a base final com 100 linhas
dados = []

for _ in range(num_linhas):
    produto = random.choice(produtos_unicos)
    item = {
        "nome_produto": produto["nome_produto"],
        "ean": produto["ean"],
        "posicao_estoque": gerar_posicao(),
        "quantidade": random.randint(1, 100),
        "preco_únitario": produto["preco"],
        "categoria": produto["categoria"],
        "data_cadastro": fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d'),
    }
    dados.append(item)

# Convertendo para DataFrame
df = pd.DataFrame(dados)

# Exibindo os primeiros registros
print(df.head())


      nome_produto            ean posicao_estoque  quantidade  preco_únitario  \
0     Draw Provide  9578108905806         G-03-13          73           90.96   
1  Apply Scientist  0947279855507         D-01-13          88           75.88   
2      Country Ago  0313722379638         D-02-08          77           43.94   
3     Before Agent  2055344305197         E-03-07          76           20.97   
4     Last Popular  0079351256190         B-04-05           8           82.91   

   categoria data_cadastro  
0  Alimentos    2024-10-16  
1    Bebidas    2024-09-11  
2    Bebidas    2025-04-06  
3    Limpeza    2024-12-18  
4    Higiene    2024-09-25  


In [ ]:
# Passo 1: Calcular o valor total por produto
df['valor_total'] = df['quantidade'] * df['preco_únitario']
valor_total_por_produto = df.groupby('nome_produto')['valor_total'].sum().reset_index()

# Passo 2: Ordenar do maior para o menor
valor_total_por_produto = valor_total_por_produto.sort_values(by='valor_total', ascending=False).reset_index(drop=True)

# Passo 3: Calcular o percentual acumulado
valor_total_por_produto['percentual'] = 100 * valor_total_por_produto['valor_total'].cumsum() / valor_total_por_produto['valor_total'].sum()

# Passo 4: Classificar como A, B ou C
def classificar(percentual):
    if percentual <= 80:
        return 'A'
    elif percentual <= 95:
        return 'B'
    else:
        return 'C'

valor_total_por_produto['curva_abc'] = valor_total_por_produto['percentual'].apply(classificar)

# Passo 5: Juntar de volta no dataframe original
df = df.merge(valor_total_por_produto[['nome_produto', 'curva_abc']], on='nome_produto', how='left')

# Resultado final
print(df.head())


          nome_produto            ean posicao_estoque  quantidade  \
0            To Forget  4722016370866         E-02-12          81   
1       Clearly Factor  8209465873705         F-02-05          85   
2        Quality Range  3340747593067         C-03-09          29   
3  Focus International  9063626809601         G-04-13          19   
4            Type Tend  0116681113196         D-03-03          24   

   preco_únitario   categoria data_cadastro  valor_total curva_abc  
0           13.02  Utilidades    2025-04-29      1054.62         C  
1           75.58  Utilidades    2025-04-15      6424.30         A  
2           69.49     Limpeza    2025-01-24      2015.21         A  
3           92.26     Limpeza    2024-12-24      1752.94         A  
4           53.92   Alimentos    2024-11-16      1294.08         A  


In [ ]:
import pandas as pd
df.to_csv('Base Galpao faker.csv', index=False, encoding='utf-8-sig',sep=';',decimal=',')

In [ ]:
df.head(100)

,nome_produto,ean,posicao_estoque,quantidade,preco_únitario,categoria,data_cadastro
0,Draw Provide,9578108905806,G-03-13,73,90.96,Alimentos,2024-10-16
1,Apply Scientist,0947279855507,D-01-13,88,75.88,Bebidas,2024-09-11
2,Country Ago,0313722379638,D-02-08,77,43.94,Bebidas,2025-04-06
3,Before Agent,2055344305197,E-03-07,76,20.97,Limpeza,2024-12-18
4,Last Popular,0079351256190,B-04-05,8,82.91,Higiene,2024-09-25
...,...,...,...,...,...,...,...
95,Win Yeah,6787128828993,E-01-01,18,22.86,Utilidades,2025-04-04
96,Resource Popular,9623471838258,A-03-01,100,13.31,Alimentos,2024-06-01
97,Bar Involve,6468815503798,F-03-02,9,77.58,Alimentos,2025-03-25
98,Make Yeah,9682856903844,A-02-01,79,22.35,Alimentos,2025-05-04
